# 03. 모델 학습 & 평가

여러 분류 모델을 학습하고 교차검증을 통해 성능을 비교한다.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import joblib

from src.evaluation import evaluate_model, plot_confusion_matrix

%matplotlib inline

## 3.1 전처리된 데이터 로드

## 3.2 베이스라인 모델

## 3.3 다중 모델 교차검증 비교

## 3.4 하이퍼파라미터 튜닝

## 3.5 최종 모델 평가

## 3.6 모델 저장